In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

/opt/conda/envs/mchatbot/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import langchain
import pinecone

print("LangChain version:", langchain.__version__)
print("Pinecone client version:", pinecone.__version__)


LangChain version: 0.2.5
Pinecone client version: 4.1.1


In [10]:
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    document = loader.load()
    return document

In [11]:
extracted_data = load_pdf("/root/src/AI_MEDICAL_CHATBOT/data/Medical_book.pdf")

In [14]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [15]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [16]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

/opt/conda/envs/mchatbot/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [18]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [20]:
from dotenv import load_dotenv
import os
load_dotenv()
key=os.getenv("PINECONE_API_KEY") 

In [21]:
key

'aa0c6b5f-16cf-4fd9-8b32-e1ad555a3382'

In [22]:
import os
from pinecone import Pinecone, ServerlessSpec

In [23]:
pc=Pinecone(api_key=key, environment='us-east-1')

In [24]:
pc

In [25]:
pc.list_indexes().names()

['medical-chatbot', 'fluxdatabase']

In [26]:
index_name = "medical-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )


In [27]:
print(index_name)

medical-chatbot


In [28]:
from langchain.vectorstores import Pinecone as LangchainPinecone

In [29]:
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.embeddings import OpenAIEmbeddings  # or the embedding model you are using
import pinecone

In [30]:
# Create the document search index
docsearch = LangchainPinecone.from_texts(
    texts=[t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name=index_name
)

In [31]:
docsearch #this is a vector database

In [32]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['Pinecone', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x782688d747c0>)

In [33]:
query = "What is fever"

In [34]:
docs=docsearch.similarity_search(query, k=3)

In [35]:
print("Result", docs)

Result [Document(page_content='fevers (a source of its nickname, “undulant fever”) can beexhausting. Symptoms usually appear between five daysand a month after exposure and begin with a single boutof high fever accompanied by shivering, aching, anddrenching sweats that last for a few days. Other symp-toms may include headache , poor appetite, backache,\nweakness, and depression. Mental depression can be sosevere that the patient may become suicidal.\nIn rare, untreated cases, the disease can become so'), Document(page_content='fevers (a source of its nickname, “undulant fever”) can beexhausting. Symptoms usually appear between five daysand a month after exposure and begin with a single boutof high fever accompanied by shivering, aching, anddrenching sweats that last for a few days. Other symp-toms may include headache , poor appetite, backache,\nweakness, and depression. Mental depression can be sosevere that the patient may become suicidal.\nIn rare, untreated cases, the disease can b

In [36]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [37]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [38]:
import requests
import os

# Create the directory if it doesn't exist
model_directory = r"/root/src/AI_MEDICAL_CHATBOT/model"

os.makedirs(model_directory, exist_ok=True)

# Hugging Face repository and file path
repo_id = "TheBloke/Llama-2-7B-Chat-GGML"
filename = "llama-2-7b-chat.ggmlv3.q4_0.bin"
download_url = f"https://huggingface.co/{repo_id}/resolve/main/{filename}"

# Download the model file from Hugging Face
model_path = os.path.join(model_directory, filename)

# Make a request with streaming enabled
response = requests.get(download_url, stream=True)

# Get the total file size from the headers
total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # Size of each block in bytes

# Initialize the progress bar
from tqdm import tqdm

tqdm_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

# Download the file in chunks
with open(model_path, 'wb') as f:
    for data in response.iter_content(block_size):
        tqdm_bar.update(len(data))
        f.write(data)

tqdm_bar.close()

# Print the path where the model is saved
print(f"Model downloaded to: {model_path}")

  0%|          | 3.12M/3.79G [00:00<02:01, 31.2MiB/s]

100%|██████████| 3.79G/3.79G [01:41<00:00, 37.3MiB/s]

Model downloaded to: /root/src/AI_MEDICAL_CHATBOT/model/llama-2-7b-chat.ggmlv3.q4_0.bin


In [39]:
# Assuming 'model_path' is defined from previous code
llm = CTransformers(
    model="/root/src/AI_MEDICAL_CHATBOT/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        'max_new_tokens': 100,  # Lowered from 512 for faster response
        'temperature': 0.7,  # Slightly reduced temperature
        'top_p': 0.9,
        'repetition_penalty': 1.0
    }
)

In [39]:
# Assuming `llm` is the loaded model instance from ctransformers

# Example query or prompt
query = "Who is Mahatma Gandhi"

# Parameters for text generation
params = {
    "max_new_tokens": 50,  # Use 'max_new_tokens' to control the length
    "temperature": 0.9,
    "top_p": 0.9,
    "repetition_penalty": 1.0
}

# Generate text based on the query
outputs = llm(query, **params)  # Pass parameters as a dictionary

# Print the generated outputs
print(outputs)

/opt/conda/envs/mchatbot/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


?
 Hinweis: Mahatma Gandhi was a political and spiritual leader in India who played a significant role in the country's struggle for independence from British rule. He is known for his philosophy of nonviolent resistance, which has inspired movements for civil rights and freedom across the world.
Mahatma Gandhi was born on October 2, 1869, in Porbandar, a coastal town in the state of Gujarat, India. His


In [40]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [41]:
query = "What are Allergies"

In [42]:


# Run the query using the QA chain
response = qa.invoke({"query": query})  # Use `invoke` instead of `run`


In [43]:
print(response['result'])

Allergies are a reaction to something in the environment that is not usually harmful, but the body thinks it is dangerous. When the body detects an allergen, it produces antibodies called IgE. These antibodies attach to mast cells, which release histamine and other chemicals that cause allergic symptoms.


In [44]:
query = "What are ulcers"

In [45]:
response = qa.invoke({"query": query})  # Use `invoke` instead of `run`

In [46]:
print(response['result'])

Ulcers, or pressure sores, are tender or inflamed patches that develop when skin covering a weight-bearing part of the body is squeezed between bone and another bodypart, or a bed, chair, splint, or other hard object.
I hope this helps! Let me know if you have any other questions.


In [31]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

In [48]:
import os
key=os.environ["PINECONE_API_KEY"] 